## Importing Libraries


In [ ]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data


## Importing Data

In [ ]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)


## Defining Placeholder

In [ ]:

no_classes=10
batch_size=128

x=tf.placeholder(tf.float32,[None, 784])
y=tf.placeholder(tf.float32,[None, 10])


## Function Definitions

### Convolution Function

In [ ]:

#user defined fuction to return output after performing convolution.

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')



### Maxpooling Function


user defined fucntion to perform maxpooling after convolution and also to pass it through the activation
and reurning the final output


In [ ]:

def maxpool2d(x):
    maxpool= tf.nn.max_pool(x, ksize=[1,2,2,1] , strides=[1,2,2,1], padding='SAME')
    return (maxpool)


## Neural Network Function

In [ ]:

#now we'll define the neural network model
'''In this model we are giving what actually happens inside 
    the neural network during the feed forward step of one epoch'''

def neural_net(x):
    weights={'w_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
             'w_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
             'w_fc':tf.Variable(tf.random_normal([7*7*64, 1024])),
             'out':tf.Variable(tf.random_normal([1024, no_classes]))}                        
                    
                                          
    biases=  {'b_conv1': tf.Variable(tf.random_normal([32])),
              'b_conv2': tf.Variable(tf.random_normal([64])),
              'b_fc': tf.Variable(tf.random_normal([1024])),
              'out': tf.Variable(tf.random_normal([no_classes]))}
   

    x=tf.reshape(x, shape=[-1,28,28,1])
    
    conv1=conv2d(x, weights['w_conv1'])
    conv1=maxpool2d(conv1)
    
    conv2=conv2d(conv1, weights['w_conv2'])
    conv2=maxpool2d(conv2)
    
    fc=tf.reshape(conv2, [-1, 7*7*64])
    fc=tf.nn.relu(tf.matmul(fc, weights['w_fc'])+ biases['b_fc'])
    
    output=tf.matmul(fc, weights['out'])+ biases['out']
    
    return output


## Function for Feed Forward and BackProp

In [ ]:

'''in this function we are working with minimising cross entropy and the application of 
gradient descent after each epoch'''

def convolutional_neural_network(x):
    prediction=neural_net(x)
    #cost = tf.square(y - prediction, name="cost")
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels= y))
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    
    no_epochs=5
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        #training the network
        for epoch in range(no_epochs):
            epoch_loss=0
            for hello in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y=mnist.train.next_batch(batch_size)
                _,c=sess.run([optimizer, cost], feed_dict={x:epoch_x, y:epoch_y})#here c means cost
                epoch_loss+=c              
            print('Epoch', epoch, 'completed out of', no_epochs, 'loss:', epoch_loss)
            
            #this lin compares the data with the values or output and apply backpropogation
        correct=tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))  
        accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))


## Calling the Feedforward function for number of epochs

In [ ]:
        
convolutional_neural_network(x)       